In [108]:
import pandas as pd
import numpy as np
import sqlite3

In [109]:
con = sqlite3.connect('data.db')
cur = con.cursor()

# Schema

In [110]:
# see all the tables
res = cur.execute("SELECT name FROM sqlite_schema WHERE type='table';")
res.fetchall()

[('games',), ('sqlite_sequence',)]

In [111]:
# see all the column names in the games table
res = cur.execute('SELECT * FROM games')
list(map(lambda x: x[0], res.description))

['id',
 'Event',
 'Site',
 'Round',
 'White',
 'Black',
 'Result',
 'UTCDate',
 'UTCTime',
 'UTCDateTime',
 'WhiteElo',
 'BlackElo',
 'WhiteRatingDiff',
 'BlackRatingDiff',
 'WhiteTitle',
 'BlackTitle',
 'ECO',
 'Variant',
 'Opening',
 'TimeControl',
 'Termination',
 'Moves']

# Query examples

Select for ELO

In [155]:
query = "SELECT * FROM games WHERE BlackElo > 1980;"
res = cur.execute(query)
print(res.fetchone())

(78, 'Rated Blitz game', 'https://lichess.org/5ug0gm2j', '', 'joecasatro', '?', '0-1', '2012.12.31', '23:41:40', '2012-12-31 23:41:40', 1081, '?', '', '', '', '', 'B54', '', 'Sicilian Defense', '300+3', 'Normal', '1. e4 c5 2. Nf3 d6 3. d4 Na6 4. Bxa6 bxa6 5. dxc5 Bb7 6. cxd6 exd6 7. Nc3 Nf6 8. e5 Ng4 9. exd6 Bxd6 10. h3 Qe7+ 11. Be3 Nxe3 12. fxe3 Qxe3+ 13. Ne2 Bg3+ 14. Kf1 Qf2# 0-1')


Select for result, and use COUNT to count number of entries instead of fetching the data itself

In [113]:
query = "SELECT COUNT(*) FROM games WHERE Result = '1-0';"
res = cur.execute(query)
print(res.fetchone())

(62129,)


Select for date range (more options: https://stackoverflow.com/questions/2309227/sqlite-select-with-condition-on-date)

In [130]:
query = "SELECT * FROM games WHERE UTCDateTime BETWEEN date('2012-12-31') AND date('2013-01-05');"
res = cur.execute(query)
print(res.fetchone())

(1, 'Rated Classical game', 'https://lichess.org/j1dkb5dw', '', 'BFG9k', 'mamalak', '1-0', '2012.12.31', '23:01:03', '2012-12-31 23:01:03', 1639, 1403, '+5', '-8', '', '', 'C00', '', 'French Defense: Normal Variation', '600+8', 'Normal', '1. e4 e6 2. d4 b6 3. a3 Bb7 4. Nc3 Nh6 5. Bxh6 gxh6 6. Be2 Qg5 7. Bg4 h5 8. Nf3 Qg6 9. Nh4 Qg5 10. Bxh5 Qxh4 11. Qf3 Kd8 12. Qxf7 Nc6 13. Qe8# 1-0')


Query with multiple things, e.g. combining usernames and ELO in various ways

In [141]:
query = "SELECT COUNT(*) FROM games WHERE White = 'BFG9k' AND BlackElo > 1600;"
res = cur.execute(query)
print(res.fetchone())

(58,)


In [152]:
query = "SELECT COUNT(*) FROM games WHERE (White = 'adepa' AND BlackElo > 1600) OR (Black = 'adepa' AND WhiteElo > 1600);"
res = cur.execute(query)
print(res.fetchone())

(10,)


# Read data into dataframe for further analysis

In [156]:
query = "SELECT * FROM games WHERE UTCDateTime BETWEEN date('2012-12-31') AND date('2013-01-05');"
# query = "SELECT * FROM games WHERE (White = 'adepa' AND BlackElo > 1600) OR (Black = 'adepa' AND WhiteElo > 1600);"
df = pd.read_sql_query(query, con)

In [157]:
df[["White", "Black", "WhiteElo", "BlackElo"]]

,White,Black,WhiteElo,BlackElo
0,BFG9k,mamalak,1639,1403
1,Desmond_Wilson,savinka59,1654,1919
2,Kozakmamay007,VanillaShamanilla,1643,1747
3,Naitero_Nagasaki,800,1824,1973
4,nichiren1967,Naitero_Nagasaki,1765,1815
...,...,...,...,...
15476,rennigeb,FrereKeiser,1441,1588
15477,promiscuo,mrschifer,1612,1438
15478,bjagus,F1_ALONSO_FERRARI,1611,1845
15479,Richard_XII,kaldere,1399,1413


In [158]:
# how many rows did we get?
df.shape[0]

15481

## Cleaning and type conversion

In [159]:
# Replace ? with NaN
# (There were some ? in the BlackElo column and maybe others)
df = df.replace(to_replace="?", value=np.nan)

/var/folders/ns/cpwqr00s7y55npv2jxl63gm80000gn/T/ipykernel_31449/575962429.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(to_replace="?", value=np.nan)


In [160]:
# See the dtypes. Might want to convert some of these to more useful things.
df.dtypes

id                   int64
Event               object
Site                object
Round               object
White               object
Black               object
Result              object
UTCDate             object
UTCTime             object
UTCDateTime         object
WhiteElo           float64
BlackElo           float64
WhiteRatingDiff     object
BlackRatingDiff     object
WhiteTitle          object
BlackTitle          object
ECO                 object
Variant             object
Opening             object
TimeControl         object
Termination         object
Moves               object
dtype: object

In [161]:
# Convert Elo to float (can't do int since there are some NaNs, but float will work fine)
df["WhiteElo"] = df["WhiteElo"].astype(np.float64)
df["BlackElo"] = df["BlackElo"].astype(np.float64)

In [162]:
# Convert DateTime column to datetime type
df["UTCDateTime"] = pd.to_datetime(df["UTCDateTime"])

## Descriptive statistics etc.

In [163]:
df.describe()

,id,UTCDateTime,WhiteElo,BlackElo
count,15481.000000,15481,15470.000000,15461.000000
mean,7741.000646,2013-01-03 03:46:56.934565120,1605.200129,1597.287950
min,1.000000,2012-12-31 23:01:03,867.000000,800.000000
25%,3871.000000,2013-01-02 05:17:35,1476.000000,1462.000000
50%,7741.000000,2013-01-03 04:33:25,1601.000000,1582.000000
75%,11611.000000,2013-01-04 02:36:00,1735.000000,1738.000000
max,15491.000000,2013-01-04 23:59:21,2372.000000,2372.000000
std,4469.125213,NaN,204.300097,211.156474


In [164]:
df["Result"].value_counts()

Result
1-0        7920
0-1        7069
1/2-1/2     492
Name: count, dtype: int64